<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1LeaguePrediccter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Module

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
from contextlib import closing
from operator import attrgetter
from pprint import pprint
from subprocess import DEVNULL
from subprocess import Popen
import time
!pip install pycaret

# Team Dic

In [15]:
TeamDic = {'札幌':'276', '仙台':'238'}
TeamIDDic = {'276':'札幌', '238':'仙台'}
TeamDic['札幌']

'276'

"comparison1_team"option value="0"チーム選択"276"札幌"31219"八戸"30673"岩手"238"仙台"30310"秋田"294"山形"30674"福島"120"鹿島"176"水戸"30305"栃木"30103"群馬"122"浦和"199"大宮"121"千葉"132"柏"270"FC東京"123"東京Ｖ"30532"町田"86"川崎Ｆ"124"横浜FM"296"横浜FC"30675"YS横浜"130"湘南"30676"相模原"150"甲府"30528"松本"30535"長野"193"新潟"30308"富山"30526"金沢"126"清水"131"磐田"30677"藤枝"31296"沼津"127"名古屋"30302"岐阜"134"京都"128"Ｇ大阪"133"Ｃ大阪"136"神戸"30000"鳥取"30314"岡山"129"広島"30851"山口"30536"讃岐"30116"徳島"30148"愛媛"30974"今治"135"福岡"30313"北九州"269"鳥栖"30531"長崎"30303"熊本"207"大分"31508"宮崎"31042"鹿児島"30317"琉球

In [6]:
# 札幌 276
# 仙台 238 
def scrape(homeTeamID, awayTeamID, year):
    data = pd.DataFrame()
    
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #チャンスビルディングポイント
    df = pd.read_html(url)[12]
    df = df.drop([0, 2, 4,6,8,10,12,14,16]).drop(df.columns[[0,1,5,6]], axis=1)
    
    # #チームスタイル
    #指数
    # df2 = pd.read_html(url)[13]
    # df2 = df2.drop([0,2,4,6,8,10,12,14]).drop(df2.columns[[0,1,5,6]], axis=1)
    #チャンス構築率
    df3 = pd.read_html(url)[16]
    df3 = df3.drop([0,2,4,6,8,10,11,13,15,17,19]).drop(df3.columns[[0,1,5,6]], axis=1)
    #ゴール期待値
    df4 = pd.read_html(url)[17]
    df4 = df4.drop([0,2,4,6]).drop(df4.columns[[0,1,5,6]], axis=1)
    #トラッキングデータ
    df5 = pd.read_html(url)[20]
    df5 = df5.drop([0,2,4,6]).drop(df5.columns[[0,1,5,6]], axis=1)
    #他のスタッツ
    df6 = pd.read_html(url)[35]
    df6 = df6.drop([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34]).drop(df6.columns[[0,1,5,6]], axis=1)

    df = pd.concat([df,df3,df4,df5,df6])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'試合平均': 'homeTeamData','Unnamed: 3': 'Data','試合平均.1': 'awayTeamData'})
    df = df[['Data', 'homeTeamData', 'awayTeamData']]

    html = requests.get(url)
    html.encoding = "UTF-8"
    soup = BeautifulSoup(html.text, "html.parser")

    texts = soup.find('div', attrs={'class': 'gameTbl'}).text
    texts = re.findall(r'\w+', texts)
    
    return df.T

In [7]:
scrape('276', '238', '2021')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44
Data,攻撃,パス,クロス,ドリブル,シュート,ゴール,奪取,守備,セーブ,攻撃回数,シュート,チャンス構築率,ゴール,成功率,被攻撃回数,被シュート,被チャンス構築率,被ゴール,被成功率,ゴール期待値,ゴール,被ゴール期待値,被ゴール,総移動距離,相手との移動距離差,スプリント数,相手とのスプリント数差,ゴール,シュート,枠内シュート,パス,クロス,直接ＦＫ,間接ＦＫ,ＣＫ,スローイン,ドリブル,タックル,クリア,インターセプト,オフサイド,警告,退場,３０ｍライン進入,ペナルティエリア進入
homeTeamData,17.83,14.01,1.77,2.05,7.58,3.39,106.54,11.87,0.41,119.6,14.6,12.2%,1.2,8.5%,120.4,11.0,9.1%,1.3,12.2%,1.544,1.2,1.435,1.3,113.451km,-2.284km,170.7,-14.5,1.2,14.6,4.2,533.7,15.4,9.4,0.7,6.0,21.3,15.3,19.4,16.7,1.4,2.0,1.0,0.1,42.8,14.9
awayTeamData,11.29,8.71,1.59,0.99,6.23,2.42,91.51,16.57,0.64,119.2,10.8,9.0%,0.9,8.0%,120.0,14.3,11.9%,1.6,11.3%,1.014,0.9,1.589,1.6,116.212km,0.619km,169.9,-10.9,0.9,10.8,3.7,392.3,12.8,9.3,1.5,3.9,23.1,10.9,18.5,25.6,2,0.9,1.2,0,30.5,9.1


In [100]:
# 札幌 276
# 仙台 238 
def scrape(homeTeamID, awayTeamID, year):
    data = pd.DataFrame()
    
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #チーム毎の平均スタッツを取得
    #チャンスビルディングポイント
    df = pd.read_html(url)[12]
    df = df.drop([0, 2, 4,6,8,10,12,14,16]).drop(df.columns[[0,1,5,6]], axis=1)
    
    # #チームスタイル
    #指数
    # df2 = pd.read_html(url)[13]
    # df2 = df2.drop([0,2,4,6,8,10,12,14]).drop(df2.columns[[0,1,5,6]], axis=1)
    #チャンス構築率
    df3 = pd.read_html(url)[16]
    df3 = df3.drop([0,2,4,6,8,10,11,13,15,17,19]).drop(df3.columns[[0,1,5,6]], axis=1)
    #ゴール期待値
    df4 = pd.read_html(url)[17]
    df4 = df4.drop([0,2,4,6]).drop(df4.columns[[0,1,5,6]], axis=1)
    #トラッキングデータ
    df5 = pd.read_html(url)[20]
    df5 = df5.drop([0,2,4,6]).drop(df5.columns[[0,1,5,6]], axis=1)
    #他のスタッツ
    df6 = pd.read_html(url)[35]
    df6 = df6.drop([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34]).drop(df6.columns[[0,1,5,6]], axis=1)

    df = pd.concat([df,df3,df4,df5,df6])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'試合平均': 'home','Unnamed: 3': 'Data','試合平均.1': 'away'})
    df = df[['Data', 'home', 'away']]
    df = df.set_index('Data')
    df = df.T

    #試合結果を取得
    html = requests.get(url)
    html.encoding = "UTF-8"
    soup = BeautifulSoup(html.text, "html.parser")

    texts = soup.find('div', attrs={'class': 'gameTbl'}).text
    texts = re.findall(r'\w+', texts)

    # #最新試合情報取得の場合
    # if (texts[0] == year):
    #     matchDate = texts[0] + '-' + texts[1] + '-' + texts[2] + '-' + texts[3] + ':' + texts[4]

    #     if (homeTeamID !=TeamDic[texts[7]]):
    #         homeGoal = texts[6]
    #         homeTeam = texts[8]
    #         awayGoal = texts[5]
    #         awayTeam = texts[7]
    #     else:
    #         homeGoal = texts[5]
    #         homeTeam = texts[7]
    #         awayGoal = texts[6]
    #         awayTeam = texts[8]
    #2試合前の情報取得の場合
    if (texts[10] == year):
        matchDate = texts[10] + '-' + texts[11] + '-' + texts[12] + '-' + texts[13] + ':' + texts[14]

        if (homeTeamID !=TeamDic[texts[17]]):
            homeGoal = texts[16]
            homeTeam = texts[18]
            awayGoal = texts[15]
            awayTeam = texts[17]
        else:
            homeGoal = texts[15]
            homeTeam = texts[17]
            awayGoal = texts[16]
            awayTeam = texts[18]
    #3試合前の情報
    elif (texts[20] == year):
        matchDate = texts[20] + '-' + texts[21] + '-' + texts[22] + '-' + texts[23] + ':' + texts[24]

        if (homeTeamID !=TeamDic[texts[27]]):
            homeGoal = texts[26]
            homeTeam = texts[28]
            awayGoal = texts[25]
            awayTeam = texts[27]
        else:
            homeGoal = texts[25]
            homeTeam = texts[27]
            awayGoal = texts[26]
            awayTeam = texts[28]

    df.insert(0, 'スコア', [homeGoal,awayGoal])
    df.insert(0, 'チーム名',[homeTeam,awayTeam])
    df.insert(0, '試合日', matchDate)

    return df

In [102]:
df = scrape('276', '238', '2021')

In [103]:
df

Data,試合日,チーム名,スコア,攻撃,パス,クロス,ドリブル,シュート,ゴール,奪取,守備,セーブ,攻撃回数,シュート,チャンス構築率,ゴール,成功率,被攻撃回数,被シュート,被チャンス構築率,被ゴール,被成功率,ゴール期待値,ゴール,被ゴール期待値,被ゴール,総移動距離,相手との移動距離差,スプリント数,相手とのスプリント数差,ゴール,シュート,枠内シュート,パス,クロス,直接ＦＫ,間接ＦＫ,ＣＫ,スローイン,ドリブル,タックル,クリア,インターセプト,オフサイド,警告,退場,３０ｍライン進入,ペナルティエリア進入
home,2021-4-24-13:00,札幌,2,17.83,14.01,1.77,2.05,7.58,3.39,106.54,11.87,0.41,119.6,14.6,12.2%,1.2,8.5%,120.4,11.0,9.1%,1.3,12.2%,1.544,1.2,1.429,1.3,113.451km,-2.284km,170.7,-14.5,1.2,14.6,4.2,533.7,15.4,9.4,0.7,6.0,21.3,15.3,19.4,16.7,1.4,2.0,1.0,0.1,42.8,14.9
away,2021-4-24-13:00,仙台,1,11.54,8.92,1.61,1.02,6.16,2.31,91.03,16.4,0.63,118.6,11.0,9.3%,0.8,7.4%,119.3,14.1,11.9%,1.6,11.3%,1.038,0.8,1.548,1.6,116.225km,0.684km,169.2,-9.6,0.8,11,3.7,400.6,13.2,9.2,1.5,4,22.9,11,17.9,25.3,2,1,1.2,0,31.3,9.4


In [27]:
homeTeamID = '276'
awayTeamID = '238'
homeGoal = ''
homeTeam = ''

awayGoal = ''
awayTeam = ''
if (homeTeamID !=TeamDic[texts[7]]):
    homeGoal = texts[6]
    homeTeam = texts[8]
    awayGoal = texts[5]
    awayTeam = texts[7]
else:
    homeGoal = texts[5]
    homeTeam = texts[7]
    awayGoal = texts[6]
    awayTeam = texts[8]



'仙台'

In [35]:
matchDate = texts[0] + '-' + texts[1] + '-' + texts[2] + '-' + texts[3] + ':' + texts[4]
matchDate

'2021-7-10-19:00'

In [56]:
df = df.drop('matchDate', axis=1)

In [99]:
texts[10]

'2021'

In [58]:
df = df.insert(0, 'スコア',)

In [63]:
print(df)

None
